In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
from tqdm.notebook import tqdm
sns.set()
rcParams['figure.figsize'] = (20,10)
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from pprint import pprint
import json
from collections import defaultdict
from googletrans import Translator
import time
from datasets import Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModel, AutoConfig, pipeline
from datasets import load_dataset, Dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-3b')

In [ ]:
translator = Translator()
def translate_df(df, batch_size=1, translator=translator):
    def translate_text(text, sleep_time=2):
        if text in [''] or text is None:
            return ''
        while True:
            try:
                return translator.translate(text, src="en", dest="vi").text
            except:
                print("Requests error")
                time.sleep(sleep_time)
    
    def clean_text(text):
        return text
    
    data = defaultdict(list)
    for i in tqdm(range(0, len(df), batch_size)):
        batch_df = df.iloc[i : i + batch_size]
        temp_data = defaultdict(list)
        for col in df.columns:
            batch_df[col] = batch_df[col].apply(lambda x: clean_text(x))
            col_vals = ' ## '.join(batch_df[col].values)
            temp_data[col] = [x.strip() for x in translate_text(col_vals).split('##')]

        can_insert_batch = True
        for col in df.columns:
            if len(temp_data[col]) != len(batch_df):
                can_insert_batch = False
                break

        if can_insert_batch:
            for col in df.columns:
                data[col].extend(batch_df[col].values)
                data[f'{col}_translated'].extend(temp_data[col])
            continue
        
        # else loop through each row
        for _, row in batch_df.iterrows():
            for col in df.columns:
                text = clean_text(row[col])
                data[col].append(text)
                data[f'{col}_translated'].append(translate_text(text))
                
    df = pd.DataFrame(data)
    return df.drop(columns=[col for col in df.columns if '_translated' not in col]) 

In [ ]:
a = """Cuộc trò chuyện giữa con người và trợ lý AI.
[|Con người|] {prompt}
[|AI|] {response}
[|Con người|]"""

def get_prompt(row):
    return a.format(prompt=row['prompt'], response=row['response'])

In [ ]:
temp = pd.read_json('../../data/translated/vi_alpaca_reduced.jsonl', lines=True)
temp2 = temp.apply(lambda x: get_prompt(x), axis=1)
temp2 = pd.DataFrame(temp2)
temp2.columns = ['prompt']

In [ ]:
temp3 = pd.read_json('../../data/translated/quora_chat_data_translated.json', orient='records')

In [ ]:
temp3.to_json('../../data/translated/quora_chat_data_translated.json',lines=True,orient='records')
temp2.to_json('../../data/translated/alpaca_chat_cleaned_translated.json',lines=True,orient='records')

## Clean ShareGPT

In [ ]:
share_gpt = pd.read_json('../../data/original/ShareGPT_V3_unfiltered_cleaned_split.json')

In [ ]:
popular_programming_keywords = [
    "python",
    "java",
    "javascript",
    "c#",
    "c++",
    "typescript",
    "ruby",
    "swift",
    "kotlin",
    "php",
    "rust",
    "scala",
    "dart",
    "matlab",
    "objective-c",
    "perl",
    "lua",
    "assert",
    "async",
    "await",
    "def",
    "elif",
    "lambda",
    "nonlocal",
    "function",
    "var",
    "cuda",
    "torch",
    "code",
    "sudo",
    "bash"
]
popular_programming_languages = [
    "python",
    "java",
    "javascript",
    "typescript",
    "kotlin",
    "objective-c",
]
def check_text_not_contain_code(text):
    temp2 = text.lower()
    temp = set(temp2.split())
    for language in popular_programming_keywords:
        if language in temp:
            return False
    for language in popular_programming_languages:
        if language in temp2:
            return False
    return True

temp = share_gpt['conversations'].apply(lambda x: str(x)).apply(check_text_not_contain_code)


In [ ]:
# remove duplicate of a Dataset
from pandas import DataFrame


DataFrame().drop_duplicates(subset=['id']).reset_index(drop=True)

In [ ]:
share_gpt_no_code = share_gpt[temp][['conversations']]
share_gpt_no_code = share_gpt_no_code[share_gpt_no_code['conversations'].apply(lambda x: len(x) >= 4)]
share_gpt_no_code.shape

In [ ]:
from numpy import nan


def create_conversation(turns):
    res = "The conversation between human and AI assistant.\n"
    for turn in turns:
        if turn['from'] == 'human':
            res += "[|Human|] " + turn['value'] + "\n"
        elif turn['from'] == 'gpt':
            res += "[|AI|] " + turn['value'] + "\n"
        else:
            return nan
    return res

share_gpt_no_code_conversations = share_gpt_no_code.apply(lambda x: create_conversation(x['conversations']), axis=1)

In [ ]:
print(share_gpt_no_code_conversations.sample(1).iloc[0])

In [ ]:
share_gpt_no_code_conversations.to_json('../../data/original/share_gpt_no_code_conversations.json', orient='records', lines=True)

### MMLU

In [129]:
mmlu_zero_shot_test = Dataset.from_json('../../../data/mmlu_eval_test/zero_shot_mmlu_test.json')
mmlu_five_shot_test = Dataset.from_json('../../../data/mmlu_eval_test/five_shot_mmlu_test.json')
mmlu_zero_shot_val = Dataset.from_json('../../../data/mmlu_eval_test/zero_shot_mmlu_val.json')
mmlu_five_shot_val = Dataset.from_json('../../../data/mmlu_eval_test/five_shot_mmlu_val.json')

def mapper(example):
    text = 'The following are multiple choice questions (with answers) about'
    example['input'] = example['input'].replace(f'{text} ', f'The conversation between human and AI assistant.\n[|Human|] {text}')
    example['output'] = f'[|AI|] {example["output"]}\n[|Human|]'
    return example

mmlu_zero_shot_test = mmlu_zero_shot_test.map(mapper)
mmlu_zero_shot_val = mmlu_zero_shot_val.map(mapper)
mmlu_five_shot_test = mmlu_five_shot_test.map(mapper)
mmlu_five_shot_val = mmlu_five_shot_val.map(mapper)

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-f460ecd73abeac7d/0.0.0)
Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-f47cf53436e97d4b/0.0.0)
Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-07bf859ffaee4a15/0.0.0)
Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-773bce18e11e9f22/0.0.0)
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-f460ecd73abeac7d/0.0.0/cache-50f99576fa046b54.arrow
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-07bf859ffaee4a15/0.0.0/cache-920236c88ff1a9cb.arrow
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-f47cf53436e97d4b/0.0.0/cache-cf64e4782697708a.arrow
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-

In [144]:
temp = mmlu_five_shot_test.shuffle()[0]
print(temp['input'])
print(temp['output'])

The conversation between human and AI assistant.
[|Human|] The following are multiple choice questions (with answers) about professional accounting.

Box a nongovernmental not-for-profit organization had the following transactions during the year: Proceeds from sale of investments $80000 Purchase of property plant and equipment $10000 Proceeds from long-term debt $100000 Loss on sale of investment $5000 What amount should be reported as net cash provided by financing activities in Box's statement of cash flows?
A. $70,000
B. $75,000
C. $80,000
D. 100000
Answer: D

One hundred years ago, your great-great-grandmother invested $100 at 5% yearly interest. What is the investment worth today?
A. $13,000
B. $600
C. $15,000
D. $28,000
Answer: A

Krete is an unmarried taxpayer with income exclusively from wages. By December 31, year 1, Krete's employer has withheld $16,000 in federal income taxes and Krete has made no estimated tax payments. On April 15, year 2, Krete timely filed for an extens

In [147]:
temp = mmlu_zero_shot_test.shuffle()[0]
print(temp['input'])
print(temp['output'])

The conversation between human and AI assistant.
[|Human|] The following are multiple choice questions (with answers) about high school physics.

In Collision A, two carts collide and bounce off each other. In Collision B, a ball sticks to a rigid rod, which begins to rotate about the combined center of mass. Which of the following statements about quantities in each collision is correct?
A. Collision A: each cart experiences the same force, time of collision, and change in kinetic energy. Collision B: the ball and the rod each experience the same torque, time of collision, and change in rotational kinetic energy.
B. Collision A: each cart experiences the same force, time of collision, and change in linear momentum. Collision B: the ball and the rod each experience the same torque, time of collision, and change in angular momentum.
C. Collision A: each cart experiences the same force, time of collision, and change in kinetic energy. Collision B: the ball and the rod each experience the

In [148]:
mmlu_zero_shot_test.to_json('../../../data/mmlu_eval_test/zero_shot_mmlu_chat_test.jsonl')
mmlu_five_shot_test.to_json('../../../data/mmlu_eval_test/five_shot_mmlu_chat_test.jsonl')
mmlu_zero_shot_val.to_json('../../../data/mmlu_eval_test/zero_shot_mmlu_chat_val.jsonl')
mmlu_five_shot_val.to_json('../../../data/mmlu_eval_test/five_shot_mmlu_chat_val.jsonl')

Creating json from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

4757760

## FAQ

In [9]:
path = '../../../data/translated/all_faqs.json'
faqs = pd.read_json(path)

In [5]:
faqs

,instruction,input,output
0,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Tội giết con mới đẻ là gì?\nĐiều luật...,Người mẹ nào do ảnh hưởng nặng nề của tư tưởng...
1,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Vì làm ăn thiếu vốn nên em có vay của...,Bộ luật Hình sự quy định về tội lừa đảo chiếm ...
2,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,"Câu hỏi: Theo quy định pháp luật, trường hợp n...",Theo quy định tại khoản 1 Điều 6 Thông tư số 1...
3,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Chủ thể chịu trách nhiệm hình sự đối ...,Chủ thể của tội làm giả hồ sơ để tiêm vaccine ...
4,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Tội khai báo gian dối hoặc cung cấp t...,"Tại Điều 382, Bộ luật Hình sự năm 2015 sửa đổi..."
...,...,...,...
6695,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Hỗ trợ doanh nghiệp vay vốn trả lương...,1. Điều kiện hỗ trợ doanh nghiệp vay vốn trả l...
6696,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Tôi muốn thành lập chi nhánh doanh ng...,Có. Theo quy định tại Điều 29 Luật đấu giá tài...
6697,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Hành vi đánh nhau có hung khí\nĐiều l...,"Theo Điều 7 Nghị định 144/2021/NĐ-CP, hành vi ..."
6698,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Người làm việc theo hợp đồng lao động...,Theo quy định tại khoản 2 Điều 2 của Thông tư ...


In [6]:
template = """{instruction}\n{input}\nCâu trả lời: {output}"""
print(template.format(**faqs.sample(1).iloc[0]))

Bạn là luật sư. Hãy trả lời câu hỏi pháp luật dựa vào những điều luật được trích dẫn dưới đây. Nếu không tìm có câu trả lời thì trả lời là tôi không biết.
Câu hỏi: Quyền và lợi ích của người lao động khi làm việc theo hợp đồng lao động bị tuyên bố vô hiệu
Điều luật liên quan: Điều 10. Xử lý hợp đồng lao động vô hiệu toàn bộ do người giao kết không đúng thẩm quyền hoặc vi phạm nguyên tắc giao kết hợp đồng lao động 1. Khi hợp đồng lao động bị tuyên bố vô hiệu toàn bộ, người lao động và người sử dụng lao động ký lại hợp đồng lao động theo đúng quy định của pháp luật. 2. Quyền, nghĩa vụ và lợi ích của người lao động kể từ khi bắt đầu làm việc theo hợp đồng lao động bị tuyên bố vô hiệu cho đến khi hợp đồng lao động được ký lại thực hiện như sau: a) Nếu quyền, lợi ích của mỗi bên trong hợp đồng lao động không thấp hơn quy định của pháp luật, thỏa ước lao động tập thể đang áp dụng thì quyền, nghĩa vụ, lợi ích của người lao động được thực hiện theo nội dung hợp đồng lao động bị tuyên bố vô hiệ

In [7]:
temp = faqs['input'].apply(lambda x: not x.endswith('Điều luật liên quan: '))
faqs = faqs[temp]

In [8]:
faqs

,instruction,input,output
1,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Vì làm ăn thiếu vốn nên em có vay của...,Bộ luật Hình sự quy định về tội lừa đảo chiếm ...
6,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Thẩm quyền giải quyết tranh chấp đất ...,Giải quyết tranh chấp đất đai là hoạt động của...
8,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Nhiệm vụ quyền hạn của Uỷ ban nhân xã...,Theo quy định tại Điều 115 Luật Tổ chức Hội đồ...
15,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,"Câu hỏi: Chào anh/chị, Về việc xin cấp Phiếu l...",Chào bạn Nguyen Vu Cuong! Trung tâm Lý lịch tư...
16,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Tôi có 120m2 đất được hưởng từ di chú...,"Trường hợp bạn hỏi, chúng tôi xin được trả lời..."
...,...,...,...
6689,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Hành vi gian lận bảo hiểm y tế bị xử ...,1. Căn cứ Điều 215 Bộ luật Hình sự 2015 quy đị...
6690,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Đơn xin nghỉ việc phải gửi trước bao ...,Theo quy định tại Điều 35 Bộ luật Lao động năm...
6691,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,Câu hỏi: Tội cướp giật tài sản được coi là hoà...,Tội cướp giật tài sản hiện nay được quy định t...
6693,Bạn là luật sư. Hãy trả lời câu hỏi pháp luật ...,"Câu hỏi: Vừa qua, tôi có yêu cầu Sở Tư pháp nơ...",Theo quy định tại khoản 3 Điều 42 Luật Lý lịch...


In [ ]:
faqs['output_len'] = faqs['output'].apply(lambda x: len(x.split()))

In [ ]:
faqs['output_len'].hist(bins=100)

In [ ]:
faqs_standard = faqs[faqs['output_len'] < 200]

In [ ]:
template = """{instruction}\n{input}\nCâu trả lời: {output}"""
print(template.format(**faqs_standard.sample(1).iloc[0]))

In [ ]:
template = """{instruction}\n{input}\nCâu trả lời: {output}"""
print(template.format(**faqs.sample(1).iloc[0]))

In [ ]:
faqs_standard

In [ ]:
temp = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/training/alpaca_chat_cleaned_51k_translated.json')
temp.rename_column(temp.column_names[0], 'text')

In [ ]:
ds = Dataset.from_json('../../data/original/gpt4-instruct-similarity-0.8-dataset.json')

In [ ]:
len(ds.shuffle()[0]['response'].split())

In [ ]:
ds

In [ ]:
from datasets import Dataset, concatenate_datasets
from glob import glob

def load_dataset(folder_path, dataset_size=None):
    data = []
    file_paths = glob(folder_path + "/*.jsonl")
    for path in file_paths:
        ds = Dataset.from_json(path)
        if len(ds.column_names) != 1:
            raise ValueError("Dataset must have only one text column")
        ds = ds.rename_column(ds.column_names[0], "text")
        data.append(ds)
    if dataset_size is None:
        return concatenate_datasets(data, axis=0).shuffle()
    return concatenate_datasets(data, axis=0).shuffle().select(range(dataset_size))

In [ ]:
ds = load_dataset('../../data/training_31_7')

In [ ]:
print(ds.shuffle()[0]['text'])

In [ ]:
from glob import glob
from datasets import Dataset, concatenate_datasets
def load_dataset(folder_path, dataset_size=None):
    data = []
    for path in glob(folder_path + "/*.jsonl"):
        ds = Dataset.from_json(path)
        if len(ds.column_names) != 1:
            raise ValueError("Dataset must have only one text column")
        ds = ds.rename_column(ds.column_names[0], "text")
        data.append(ds)
    if dataset_size is None:
        final_ds = concatenate_datasets(data, axis=0).shuffle(seed=42)
    final_ds = (
        concatenate_datasets(data, axis=0).shuffle(seed=42).select(range(dataset_size))
    )
    final_ds = final_ds.filter(lambda x: x["text"] != "" or x["text"] is not None)
    return final_ds

In [ ]:
ds = load_dataset('../../data/training_31_7')

In [ ]:
df = ds.to_pandas()

In [ ]:
def check(x):
    if x == '' or x is None:
        return True
    return False
temp = df['text'].apply(check)

In [ ]:
temp.sum()

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloomz-3b')

In [ ]:
tokenizer

In [ ]:
tokenizer.tokenize('')

In [3]:
en_tree = []
with open('../../data/original/oasst_ready_trees.json') as f:
    for line in f:
        temp = json.loads(line)
        if temp['prompt']['lang'] in ['en']:
            en_tree.append(temp)

In [4]:
vi_tree = []
with open('../../data/original/oasst_ready_trees.json') as f:
    for line in f:
        temp = json.loads(line)
        if temp['prompt']['lang'] in ['vi']:
            vi_tree.append(temp)

In [5]:
def traverse(root):
    if len(root['replies']) == 0:
        return [[(root['role'], root['text'])]]
    data = []
    for reply in root['replies']:
        data.extend(traverse(reply))
    data = [[(root['role'], root['text'])] + d for d in data]
    return data

In [6]:
en_conversations = []
for tree in en_tree:
    en_conversations.extend(traverse(tree['prompt']))

vi_conversations = []
for tree in vi_tree:
    vi_conversations.extend(traverse(tree['prompt']))

In [7]:
len(en_conversations)

21164

In [8]:
len(vi_conversations)

129

In [15]:
templates = "Cuộc trò chuyện giữa con người và trợ lý AI.\n{conversation}"
def format(conversations):
    data = []
    for conversation in conversations:
        temp = ""
        for turn in conversation:
            if turn[0] == 'prompter':
                temp += f"[|Con người|] {turn[1]}\n"
            else:
                temp += f"[|AI|] {turn[1]}\n"
        data.append(templates.format(conversation=temp))
    return data

In [11]:
en_data = format(en_conversations)

In [16]:
vi_data = format(vi_conversations)

In [33]:
print(en_data[11])

The conversation between human and AI assistant.
[|Human|] Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
[|AI|] Can you write a formal letter to introduce Jeff Bezos to a customer?
[|Human|] I asked about contrastive learning in machine learning which has nothing to do with Jeff Bezos. Contrastive learning is used to increase the performance of vision-based tasks using contrast. I want you to explain this technique to me in a way that anyone without machine learning knowledge would understand.
[|AI|] In simple terms, contrastive learning focuses on teaching an AI the points of similarity between different images (or other media) to indirectly aid it in spotting points of divergence when present. To anthropomorphize the process, a human engaged in contrastive learning and eating hundreds of apples in a week would be better equipped to recognize an orange when presented with one.



In [24]:
en_df = pd.DataFrame(en_data, columns=['text'])
en_df.to_json('../../data/original/en_oasst.json', orient='records', lines=True)

WiVi

In [648]:
wivi = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/wizard_vicuna_dataset_v2.json')
def mapper(x):
    conversation = ""
    for turn in x['conversations']:
        if turn['from'] == 'gpt':
            conversation += f"[|AI|] {turn['value']}\n" 
        elif turn['from'] == 'human':
            conversation += f"[|Human|] {turn['value']}\n"
        else:
            raise Exception()
    x['conversations'] = conversation.strip()
    return x


popular_programming_keywords = [
    "python",
    "java",
    "javascript",
    "c#",
    "c++",
    "typescript",
    "ruby",
    "swift",
    "kotlin",
    "php",
    "rust",
    "scala",
    "dart",
    "matlab",
    "objective-c",
    "perl",
    "elif",
    "lambda",
    "nonlocal",
    "function",
    "cuda",
    "torch",
    "code",
    "sudo",
    "bash",
    "int",
    "html",
    "main()",
    "chinese",
    '\begin'
]
popular_programming_languages = [
    "sql",
    "linux",
    "</",
    "/>",
    "bash",
    "python",
    "java",
    "javascript",
    "typescript",
    "swift",
    "kotlin",
    "rust",
    "scala",
    "dart",
    "matlab",
    'latex',
    '\begin'
]
def check_text_not_contain_code(text):
    temp2 = text.lower()
    temp = set(temp2.split())
    for language in popular_programming_keywords:
        if language in temp:
            return False
    for language in popular_programming_languages:
        if language in temp2:
            return False
    return True

wivi_no_code = wivi.map(mapper).filter(lambda x: check_text_not_contain_code(x['conversations']))
wivi_no_code

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-40bcae9625fbbf4b/0.0.0)


Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-40bcae9625fbbf4b/0.0.0/cache-303a7531d5d40b69.arrow


Filter:   0%|          | 0/70000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'conversations'],
    num_rows: 30527
})

In [649]:
print(wivi_no_code.shuffle()[0]['conversations'])

[|Human|] Create a slogan that describes the company's values or mission.
Company Name: Comfort Now
[|AI|] "Bringing you comfort, one step at a time."
[|Human|] Can you explain how Comfort Now provides comfort to their customers?
[|AI|] Comfort Now provides comfort to their customers through a range of products and services. They offer HVAC systems, air conditioning units, heating systems, and indoor air quality solutions. All of these products are designed to keep customers comfortable in their homes or places of business. In addition to providing top-quality products, Comfort Now also offers exceptional customer service, ensuring that their clients have the support they need to stay comfortable year-round.
[|Human|] What makes Comfort Now's HVAC systems and air conditioning units stand out from other brands?
[|AI|] Comfort Now's HVAC systems and air conditioning units stand out from other brands because they are designed with the latest technology and energy-efficient features. Their

In [650]:
wivi_no_code.to_json('../../../data/original/wizard_vicuna_nocode.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

126218749

### Wizard

In [446]:
wizard = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/wizard_full.jsonl')
def mapper(x):
    conversation = ""
    for turn in x['conversations']:
        if turn['from'] == 'gpt':
            conversation += f"[|AI|] {turn['value']}\n" 
        elif turn['from'] == 'human':
            conversation += f"[|Human|] {turn['value']}\n"
        else:
            raise Exception()
    x['conversations'] = conversation.strip()
    return x


popular_programming_keywords = [
    "python",
    "java",
    "javascript",
    "c#",
    "c++",
    "typescript",
    "ruby",
    "swift",
    "kotlin",
    "php",
    "rust",
    "scala",
    "dart",
    "matlab",
    "objective-c",
    "perl",
    "elif",
    "lambda",
    "nonlocal",
    "function",
    "cuda",
    "torch",
    "code",
    "sudo",
    "bash",
    "int",
    "html",
    "main()",
    "chinese",
    '\begin',
]
popular_programming_languages = [
    "sql",
    "linux",
    "</",
    "/>",
    "bash",
    "python",
    "java",
    "javascript",
    "c#",
    "c++",
    "typescript",
    "swift",
    "kotlin",
    "rust",
    "scala",
    "dart",
    "matlab",
    'latex'
    '\begin',
]
def check_text_not_contain_code(text):
    temp2 = text.lower()
    temp = set(temp2.split())
    for language in popular_programming_keywords:
        if language in temp:
            return False
    for language in popular_programming_languages:
        if language in temp2:
            return False
    return True

wizard_no_code = wizard.map(mapper).filter(lambda x: check_text_not_contain_code(x['conversations']))
wizard_no_code

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-e3dcd79dfa896dc5/0.0.0)
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-e3dcd79dfa896dc5/0.0.0/cache-a39fe876423bddf1.arrow
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-e3dcd79dfa896dc5/0.0.0/cache-d4e1c884044eb18e.arrow


Dataset({
    features: ['idx', 'conversations'],
    num_rows: 82130
})

In [451]:
print(wizard_no_code.shuffle()[0]['conversations'])

[|Human|] From a given set of 50 semiconductor diodes with different forward and reverse bias values, and a power source with an output of 12V and a maximum current flow of 3A, calculate the individual currents flowing through each diode under different operating conditions such as temperature, humidity, and atmospheric pressure. Additionally, predict and identify any effects of aging and wear on the diodes' performance and stability, including any thermal dissipation, voltage drop, and noise generation. Finally, propose any corrective measures that may be necessary to ensure the optimal and reliable operation of the system over a longer lifetime.
[|AI|] To calculate the individual currents flowing through each diode under different operating conditions, we need to use the diode's forward and reverse characteristics and the power source's voltage and current limits. The diode's forward characteristics can be obtained from its datasheet or through testing, while the reverse characterist

In [762]:
path = '/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/wizard_no_code.jsonl'
wizard_no_code.to_json(path, orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/83 [00:00<?, ?ba/s]

171545449

In [676]:
'\begin' in """[|Human|] Can you provide a list of 3 popular tourist attractions in Tokyo? 
\begin{itemize}
    \item The first tourist attraction is the Tokyo Tower, which is a communications and observation tower located in the Shiba-koen district of Minato, Tokyo. It stands at a height of 333 meters, making it one of the tallest structures in Japan.
    \item The second tourist attraction is the Sensoji Temple, which is a Buddhist temple located in the Asakusa district of Tokyo. It is one of the oldest and most famous temples in Tokyo, and it attracts millions of visitors every year.
    \item The third tourist attraction is the Meiji Shrine, which is a Shinto shrine located in the Shibuya district of Tokyo. It is dedicated to the deified spirits of Emperor Meiji and Empress Shoken, and it is one of the most popular shrines in Tokyo.
\end{itemize}
[|AI|] These are the 3 popular tourist attractions in Tokyo:
\begin{enumerate}
    \item Tokyo Tower
    \item Sensoji Temple
    \item Meiji Shrine
\end{enumerate}""".lower()

True

### Okapi

In [434]:
okapi_instruct = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/okapi/okapi_instruct_vi.json')
okapi_rm = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/okapi/okapi_rm_vi.json')
okapi_rl = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/okapi/okapi_rl_vi.json')
okapi_rl = okapi_rl.rename_column('prefered_output','output')
full_okapi =  concatenate_datasets([
    okapi_instruct.select_columns(['instruction', 'input', 'output']),
    okapi_rm.select_columns(['instruction', 'input', 'output']),
    okapi_rl.select_columns(['instruction', 'input', 'output'])
])
full_okapi = Dataset.from_pandas(full_okapi.to_pandas().drop_duplicates(subset=['output']).reset_index(drop=True))
print(okapi_instruct.shape, okapi_rm.shape, okapi_rl.shape, full_okapi.shape)
full_okapi.shuffle()[0]

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-794bec094f7dc439/0.0.0)
Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-ad47e4e965b3d8a9/0.0.0)
Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-3bfdc3cc940add2c/0.0.0)


(51770, 6) (64776, 4) (126675, 5) (155002, 3)


{'instruction': 'Đề xuất một cách mà người nói có thể thay đổi thái độ của mình.',
 'input': 'Người nói đang cảm thấy thiếu động lực và cảm hứng.',
 'output': 'Người nói có thể thử thay đổi góc nhìn bằng cách nhìn vào tình huống đó từ một góc nhìn khác hoặc bằng cách tạo ra một kế hoạch hoặc mục tiêu mới.'}

In [443]:
key_words = [
    "code"
    "python",
    "java",
    "javascript",
    "c#",
    "c++",
    "typescript",
    "ruby",
    "swift",
    "kotlin",
    "php",
    "rust",
    "scala",
    "dart",
    "matlab",
    "objective-c",
    "perl",
    "elif",
    "lambda",
    "nonlocal",
    "function",
    "def",
    "html","css",
    "sql",
    "bash",
    'latex',
    "print",
    "import",
    "return",
    "from"
]
def check_text_contain_code(text):
    if text is None or text == '':
        return False
    temp = text.lower()
    for kw in key_words:
        if kw in temp:
            return True
    return False

def filter_code(x):
    return check_text_contain_code(x['input']) or check_text_contain_code(x['output'])

code_okapi = full_okapi.filter(filter_code)
code_okapi

Filter:   0%|          | 0/155002 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5973
})

In [444]:
temp = code_okapi.shuffle()[0]
print(temp['instruction'] + '\n----\n' + temp['input'] + '\n----\n' + temp['output'])

Cho một đoạn mã đầu vào, sửa đổi nó sao cho nó xuất ra tổng bình phương của các số từ 1 đến 10.
----
# Xuất tổng của các số từ 1 đến 10
for i in range(1,11):
  print(i)
----
# Xuất tổng bình phương của các số từ 1 đến 10
total = 0
for i in range(1,11):
  total += i*i
print(total)


In [400]:
no_code_okapi = full_okapi.filter(lambda x: not check_text_contain_code(x['input']) and not check_text_contain_code(x['output']))

Filter:   0%|          | 0/158035 [00:00<?, ? examples/s]

In [401]:
no_code_okapi

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 152021
})

In [332]:
temp = full_okapi.shuffle()[0]
print(temp['instruction'] + '\n----\n' + temp['input'] + '\n----\n' + temp['output'])

Tạo ra một câu chuyện về cách hai người gặp nhau?
----

----
Họ gặp nhau vào một ngày hè ấm áp trong khi đi dạo trong công viên. Cô đang đi dạo trên đường đi bộ, tận hưởng ánh nắng mặt trời và cơn gió mát thổi qua những cây cối. Anh đã đến công viên cùng chú chó của mình để đi dạo, nhưng cho đến hiện tại, anh chỉ thấy được những con sóc và chim. Khi anh đi quanh một góc, anh nhìn thấy cô. Cô đang mặc một chiếc váy hè đơn giản và nhìn lên cây, trông như một vật thế giới khác trong ánh nắng. Anh cảm thấy bị thu hút bởi cô ấy. Anh hỏi cô ấy có muốn đi dạo cùng anh không, và cô ấy đồng ý. Từ khoảnh khắc đó trở đi, họ trở nên không thể tách rời, cuối cùng kết hôn và có một gia đình của riêng mình.


In [445]:
# full_okapi.to_json('../../../data/original/okapi/full_okapi.jsonl', orient='records', lines=True)
code_okapi.to_json('../../../data/original/okapi/code_related_okapi.jsonl', orient='records', lines=True)
# no_code_okapi.to_json('../../../data/original/okapi/no_code_okapi.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

4123023